In [18]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle

DIRECTORY = 'data'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [19]:
def load_data():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    names = [n.decode('utf-8')
             for n in unpickle(DIRECTORY + "/batches.meta")[b'label_names']]
    x_train = None
    y_train = []

    for i in range(1, 6):
        data = unpickle(DIRECTORY + "/data_batch_" + str(i))
        if i > 1:
            x_train = np.append(x_train, data[b'data'], axis=0)
        else:
            x_train = data[b'data']
        y_train += data[b'labels']

    data = unpickle(DIRECTORY + "/test_batch")

    x_test = data[b'data']
    y_test = data[b'labels']

    return names, x_train, y_train, x_test, y_test


In [20]:
def plot_tensor(tensor, perm=None):
    if perm == None:
        perm = (1, 2, 0)
    plt.figure()
    plt.imshow(tensor.permute(perm).cpu().numpy().astype(np.uint8))
    plt.show()


In [21]:
names, x_train, y_train, x_test, y_test = load_data()
names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [22]:


x_train_tensors = torch.tensor(x_train).to(device)
x_train_tensors = torch.reshape(x_train_tensors, [50000, 3, 32, 32])
x_test_tensors = torch.tensor(x_train).to(device)
x_test_tensors = torch.reshape(x_test_tensors, [50000, 3, 32, 32])

x_test_tensors.shape


torch.Size([50000, 3, 32, 32])

In [23]:



my_cnn = torch.nn.Conv2d(3, 1, (3, 3)).to(device)

output = my_cnn(x_train_tensors.float())

In [24]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

class Net(Module):
  def __init__(self, num_channels, classes) -> None:
    super(Net, self).__init__()
    
    # conv_layer -> relu -> pool
    self.conv_1 = Conv2d(in_channels=num_channels, out_channels=20, kernel_size=(5, 5))
    self.relu_1 = ReLU()
    self.maxpool_1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

    # second set
    # conv -> relu -> pool
    self.conv_2 = Conv2d(in_channels=20, out_channels=50, kernel_size=(5, 5))
    self.relu_2 = ReLU()
    self.maxpool_2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

    # initializa fully connected layer
    self.fc_1 = Linear(in_features=800, out_features=500)
    self.relu_3 = ReLU()

    # initialize softmax classifier
    self.fc_2 = Linear(in_features=500, out_features=classes)
    self.log_soft_max = LogSoftmax(dim=1)

  def forward(self, x):
    # the x data passes through our first set 
    # conv -> relu -> pool layers
    x = self.conv_1(x)
    print(x.shape)
    x = self.relu_1(x)
    x = self.maxpool_1(x)

    # passes the output from previous layer to through the second
    # conv -> relu -> pool

    x = self.conv_2(x)
    x = self.relu_2(x)
    x = self.maxpool_2(x)

    # flatten the output and passes it
    # fc -> relu
    x = flatten(x, 1)
    x = self.fc_1(x)
    x = self.relu_3(x)

    # pass the output to our softmax to get our final output
    x = self.fc_2(x)
    return self.log_soft_max(x)

In [25]:
model = Net(num_channels=3, classes=len(names))
model.to(device)

for e in range(10):
  model.train()

  for i in range(len(x_train_tensors)):
    pred = model(x_train_tensors[i].float())
    # input()

torch.Size([20, 28, 28])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (50x25 and 800x500)

In [ ]:
pred